In [2]:
import torch

import sys
from pathlib import Path
import os
assert "HF_TOKEN" in os.environ
project_root = Path().resolve().parent
sys.path.append(str(project_root))


from src.evaluation.evaluate import evaluate_and_plot, evaluate_benchmark
from src.methods.fewshot.sampling import sample_k_per_class, sample_fewshot_task
from src.methods.fewshot.prototypical import compute_prototypes, classify
from transformers import HubertModel
from src.datasets.snoring_dataset import SnoringDataset
from src.methods.fewshot.benchmark import benchmark_fewshot

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
root_dir = "Snoring Dataset"

dataset = SnoringDataset(root_dir)

print("Classes :", dataset.classes)
print("Nombre total de fichiers :", len(dataset))


100%|██████████| 113M/113M [00:23<00:00, 4.96MB/s] 

Extracting files...


Classes : ['0', '1']
Nombre total de fichiers : 1000


In [4]:
X = []
y = []

for waveform, label in dataset:
    X.append(waveform)
    y.append(label)

X = torch.stack(X)      
y = torch.tensor(y)     


print(X.shape)  
print(y.shape)  


torch.Size([1000, 16000])
torch.Size([1000])


In [5]:
X_support, y_support, X_query, y_query = sample_k_per_class(
    X, y, k=1
)

In [6]:
hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960", use_safetensors=True)
hubert.eval().to(device)

for p in hubert.parameters():
    p.requires_grad = False

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

In [7]:
prototypes, proto_labels = compute_prototypes(
    hubert, X_support, y_support
)

preds = classify(
    hubert, X_query, prototypes, proto_labels
)

acc = (preds.cpu() == y_query).float().mean()
print("Few-shot accuracy:", acc.item())


KeyboardInterrupt: 

## Benchmark

In [ ]:
results = {}
for k in [1,5,10]:
    results[k] = benchmark_fewshot(model=hubert, X=X, y=y, device=device, k_shot=k, n_tasks=10)


1-shot benchmark:  90%|█████████ | 9/10 [00:48<00:06,  6.73s/it]

In [ ]:
evaluate_benchmark(results, class_names=dataset.classes)